Baseado em https://github.com/nzw0301/keras-examples/blob/master/NNLM.ipynb

In [179]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import numpy as np
np.random.seed(13)

In [180]:
#path = get_file('alice.txt', origin='http://www.gutenberg.org/cache/epub/11/pg11.txt')
path = get_file('salmos.txt', origin='http://www.gutenberg.org/cache/epub/11/pg11.txt')
doc = open(path).readlines()[3:10]

In [181]:
doc[:10]

['dos maus, que não seguem o exemplo dos que não querem saber de Deus e\n',
 'que não se juntam com os que zombam de tudo o que é sagrado!\n',
 '\x7f2 Pelo contrário, o prazer deles está na lei do SENHOR, e nessa\n',
 'lei eles meditam dia e noite.\n',
 '\x7f3 Essas pessoas são como árvores que crescem na beira de um\n',
 'riacho; elas dão frutas no tempo certo, e as suas folhas não murcham.\n',
 'Assim também tudo o que essas pessoas fazem dá certo.\n']

In [182]:
tokenizer = Tokenizer()

In [183]:
tokenizer

In [184]:
tokenizer.fit_on_texts(doc)

In [185]:
doc = tokenizer.texts_to_sequences(doc)

In [186]:
doc[:10]

[[10, 21, 1, 4, 16, 3, 33, 10, 1, 4, 47, 23, 5, 15, 2],
 [1, 4, 18, 53, 55, 17, 1, 38, 5, 11, 3, 1, 24, 39],
 [34, 20, 26, 3, 58, 43, 57, 7, 9, 37, 14, 2, 49],
 [9, 40, 32, 46, 2, 27],
 [59, 8, 12, 30, 29, 54, 1, 13, 7, 48, 5, 50],
 [25, 41, 51, 28, 56, 45, 6, 2, 42, 35, 36, 4, 52],
 [19, 31, 11, 3, 1, 8, 12, 44, 22, 6]]

In [187]:
doc = [l for l in doc if len(l) > 1]

In [188]:
doc[:10]

[[10, 21, 1, 4, 16, 3, 33, 10, 1, 4, 47, 23, 5, 15, 2],
 [1, 4, 18, 53, 55, 17, 1, 38, 5, 11, 3, 1, 24, 39],
 [34, 20, 26, 3, 58, 43, 57, 7, 9, 37, 14, 2, 49],
 [9, 40, 32, 46, 2, 27],
 [59, 8, 12, 30, 29, 54, 1, 13, 7, 48, 5, 50],
 [25, 41, 51, 28, 56, 45, 6, 2, 42, 35, 36, 4, 52],
 [19, 31, 11, 3, 1, 8, 12, 44, 22, 6]]

In [189]:
words_size = sum([len(words) - 1 for words in doc])

In [190]:
words_size

76

In [191]:
maxlen = max([len(x)-1 for x in doc])

In [192]:
maxlen

14

In [193]:
vocab_size = len(tokenizer.word_index) + 1

In [194]:
vocab_size

60

In [195]:
def generate_data(X, maxlen, V):
    for sentence in X: 
        print('sentence:',sentence)
        inputs = []
        targets = []
        for i in range(1, len(sentence)):
            inputs.append(sentence[0:i])
            targets.append(sentence[i])
        y = np_utils.to_categorical(targets, V)
        inputs_sequence = sequence.pad_sequences(inputs, maxlen=maxlen)
        yield (inputs_sequence, y)

In [196]:
def sample(p):
    p /= sum(p)
    return np.where(np.random.multinomial(1, p, 1)==1)[1][0]

In [197]:
vocab_size,128,maxlen

(60, 128, 14)

In [200]:
for x_1, y_1 in generate_data(doc, maxlen, vocab_size):
    print(x_1,y_1)
    print("_________________________________________________________________________________")

sentence: [10, 21, 1, 4, 16, 3, 33, 10, 1, 4, 47, 23, 5, 15, 2]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 10 21]
 [ 0  0  0  0  0  0  0  0  0  0  0 10 21  1]
 [ 0  0  0  0  0  0  0  0  0  0 10 21  1  4]
 [ 0  0  0  0  0  0  0  0  0 10 21  1  4 16]
 [ 0  0  0  0  0  0  0  0 10 21  1  4 16  3]
 [ 0  0  0  0  0  0  0 10 21  1  4 16  3 33]
 [ 0  0  0  0  0  0 10 21  1  4 16  3 33 10]
 [ 0  0  0  0  0 10 21  1  4 16  3 33 10  1]
 [ 0  0  0  0 10 21  1  4 16  3 33 10  1  4]
 [ 0  0  0 10 21  1  4 16  3 33 10  1  4 47]
 [ 0  0 10 21  1  4 16  3 33 10  1  4 47 23]
 [ 0 10 21  1  4 16  3 33 10  1  4 47 23  5]
 [10 21  1  4 16  3 33 10  1  4 47 23  5 15]] [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [123]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=maxlen))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [97]:
for i in range(10):
    for x, y in generate_data(doc, maxlen, vocab_size):
        model.train_on_batch(x, y)

    in_words = "salmos"
    for _ in range(maxlen):
        in_sequence = sequence.pad_sequences(tokenizer.texts_to_sequences([in_words]), maxlen=maxlen)
        wordid = sample(model.predict(in_sequence)[0])
        for k, v in tokenizer.word_index.items():
            if v == wordid:
                in_words += ' ' + k
                break

    print(i, in_words)

0 salmos quebrará céu dá quando em inocente posso caminho protege armadilhas capitulo ti inchados quando vejam
1 salmos pois os furor pode meu compaixão irado capitulo afiará fim agradecerei acabes vocês pensam furor
2 salmos crianças é em meu a planos tens alguma arrependerem esquecidos os daqueles não acabemos juiz
3 salmos o eles e os lembrem protege foram olho ó completa e capitulo rebeldes não adorem
4 salmos com e livrar meus eles maus alegrarei pecar rei que acabes injustiça agora com me
5 salmos e peixes ele persigam em e me 7 me violência noite altíssimo quando no ser
6 salmos me quando céu os e ficarão oro mais tem pega meu me no senhor meus
7 salmos o 4 as capitulo não eles o lugares barra fazer senhor até mar medo seu
8 salmos a castigues alguém o salvar armadilhas ti te nosso 10 morte preparam espero deixam felizes
9 salmos eu que és contra grandeza seres pode caído encontram na céu povos essas ninguém detestas


In [201]:
in_sequence = sequence.pad_sequences(tokenizer.texts_to_sequences([in_words]), maxlen=maxlen)

In [202]:
in_sequence

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 7, 8]])

In [203]:
in_words

'salmos eu que és contra grandeza seres pode caído encontram na céu povos essas ninguém detestas'

In [204]:
in_sequence

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 7, 8]])

In [205]:
wordid

402

In [206]:
tokenizer.word_index.items()

dict_items([('crescem', 13), ('certo', 6), ('senhor', 14), ('como', 29), ('seguem', 16), ('os', 17), ('se', 18), ('assim', 19), ('pelo', 20), ('na', 7), ('essas', 8), ('e', 2), ('lei', 9), ('o', 3), ('dos', 10), ('maus', 21), ('saber', 23), ('é', 24), ('não', 4), ('riacho', 25), ('contrário', 26), ('noite', 27), ('frutas', 28), ('são', 30), ('também', 31), ('meditam', 32), ('exemplo', 33), ('dão', 51), ('\x7f2', 34), ('suas', 35), ('dia', 46), ('folhas', 36), ('do', 37), ('árvores', 54), ('sagrado', 39), ('eles', 40), ('elas', 41), ('com', 55), ('deles', 43), ('tempo', 45), ('pessoas', 12), ('tudo', 11), ('querem', 47), ('beira', 48), ('dá', 22), ('deus', 15), ('um', 50), ('fazem', 44), ('de', 5), ('nessa', 49), ('murcham', 52), ('juntam', 53), ('zombam', 38), ('as', 42), ('no', 56), ('está', 57), ('prazer', 58), ('\x7f3', 59), ('que', 1)])